<a href="https://colab.research.google.com/github/SteinwayThor/eeg-features-movies/blob/main/neural_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
# IMPORTS
import os
from google.colab import drive
import bz2
import _pickle as cPickle
import numpy as np
import pandas as pd

In [2]:
drive.mount('/content/drive', force_remount=True)

# Load any compressed pickle file
def decompress_pickle(file):
  data = bz2.BZ2File(file, 'rb')
  data = cPickle.load(data)
  return data

Mounted at /content/drive


In [15]:
neural_data_folder = '/content/drive/MyDrive/Movie Data/e0006KR/'
neural_data_dir = os.listdir(neural_data_folder)

In [37]:
#Extract start and end of chapters:
events_f = [f for f in neural_data_dir if 'events' in f.lower()][0]
events = decompress_pickle(os.path.join(neural_data_folder, events_f))
chapter_idx = [event for event in events['signal'] if event[0]==9 or event[0]==18]
start_stop = [(chapter_idx[i][1],chapter_idx[i+1][1]) for i in range(0,len(chapter_idx), 2)]
chapter_idx = dict(zip((range(1,len(start_stop)+1)),start_stop))

In [38]:
chapter_idx

{1: (23828108, 24179424),
 2: (24181586, 24557675),
 3: (24558933, 24956256),
 4: (24957529, 25294656),
 5: (25295943, 25593803),
 6: (25595013, 25965313),
 7: (25966462, 26370719),
 8: (26371896, 26711407),
 9: (26712625, 27178388),
 10: (27179752, 27695292),
 11: (27696813, 28188900),
 12: (28190842, 28532606),
 13: (28533966, 29040089),
 14: (29041771, 29418395),
 15: (29419847, 29804615),
 16: (29806140, 30300732),
 17: (30302312, 30687636),
 18: (30689435, 30901312),
 19: (30902789, 31380891),
 20: (31382496, 31449857)}

In [56]:
list(chapter_idx.keys())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [54]:
#find all channels:
channels = []
for f in neural_data_dir:
    if (not '.pbz' in f.lower()) or ('event' in f.lower()):
      continue
    channel_name = f[f.rfind('_')+1:-5]
    channels.append(channel_name)

In [69]:
processed_data = dict((i,dict((i,[]) for i in channels)) for i in chapter_idx)

In [62]:
processed_data.keys(), processed_data[1].keys()

(dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]),
 dict_keys(['RPH12', 'RPH14', 'RPH13', 'RPH11', 'RPH10', 'RPH9', 'RPH8', 'RPH7', 'RPH6', 'RPH5', 'RPH4', 'RPH3', 'RPH2', 'RPH1', 'RAH14', 'RAH13', 'RAH12', 'RAH11', 'RAH10', 'RAH9', 'RAH8', 'RAH7', 'RAH6', 'RAH5', 'RAH4', 'RAH3', 'RAH2', 'RAH1', 'RAMY14', 'RAMY13', 'RAMY12', 'RAMY11', 'RAMY10', 'RAMY9', 'RAMY8', 'RAMY7', 'RAMY6', 'RAMY5', 'RAMY4', 'RAMY3', 'RAMY2', 'RAMY1', 'LPH14', 'LPH13', 'LPH12', 'LPH11', 'LPH10', 'LPH9', 'LPH8', 'LPH7', 'LPH6', 'LPH5', 'LPH4', 'LPH3', 'LPH2', 'LPH1', 'LAH14', 'LAH13', 'LAH12', 'LAH11', 'LAH10', 'LAH9', 'LAH8', 'LAH7', 'LAH6', 'LAH5', 'LAH4', 'LAH3', 'LAH2', 'LAH1', 'LAMY14', 'LAMY13', 'LAMY12', 'LAMY11', 'LAMY10', 'LAMY9', 'LAMY8', 'LAMY7', 'LAMY6', 'LAMY5', 'LAMY4', 'LAMY3', 'LAMY2', 'LAMY1']))

In [71]:
for chapter in chapter_idx:
  print(chapter)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [68]:
# Load in neural data
# NOTE: Takes a while!
#chapter_idx
def process_channel(channel_name, channel_data):
    #chapter wise data
    for chapter in processed_data:
      processed_data[chapter][channel_name] = channel_data['signal'][chapter_idx[chapter][0]:chapter_idx[chapter][1]]
for chapter in chapter_idx:

for f in neural_data_dir:
    if (not '.pbz' in f.lower()) or ('event' in f.lower()):
      continue
    channel_name = f[f.rfind('_')+1:-5]
    channel_data = decompress_pickle(os.path.join(neural_data_folder, f))
    process_channel(channel_name, channel_data)
    del channel_data
    print(f'Processed channel {channel_name}')


Processed channel RPH12
Processed channel RPH14
Processed channel RPH13
Processed channel RPH11
Processed channel RPH10
Processed channel RPH9
Processed channel RPH8


KeyboardInterrupt: ignored